# Importing necessary libraries

In [57]:
import pandas as pd
import pm4py as pm4
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelEncoder

from scipy import stats

In [58]:
df_old = pm4.convert_to_dataframe(pm4.read.read_xes('BPI_Challenge_2012.xes.gz'))
df_old.head()

parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
1,112,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
2,112,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
3,112,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000
4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000


In [59]:
#Make a list of all possible concept_names
concept_names = ['A_ACCEPTED', 'A_ACTIVATED', 'A_APPROVED', 'A_CANCELLED', 'A_DECLINED', 'A_FINALIZED', 'A_PREACCEPTED', 'A_REGISTERED', 'O_ACCEPTED', 'O_CANCELLED', 'O_CREATED',
                 'O_DECLINED', 'O_SELECTED', 'O_SENT', 'O_SENT_BACK', 'W_Afhandelen leads', 'W_Beoordelen fraude', 'W_Nabellen incomplete dossiers', 'W_Nabellen offertes',
                   'W_Valideren aanvraag', 'W_Wijzigen contractgegevens']

In [60]:
df_old['remove'] = 0
completed_dict = {}

# Save the case:concept:name and the corresponding activity that have a complete lifecycle transition.

for line in range(len(df_old)):
    if df_old.loc[line, 'lifecycle:transition'] == 'COMPLETE' and df_old.loc[line, 'case:concept:name'] not in completed_dict.keys():
        completed_dict[df_old.loc[line,'case:concept:name']] = [df_old.loc[line, 'concept:name']]
    elif df_old.loc[line, 'lifecycle:transition'] == 'COMPLETE' and df_old.loc[line, 'case:concept:name'] in completed_dict.keys():
        completed_dict[df_old.loc[line,'case:concept:name']] += [df_old.loc[line, 'concept:name']]

# If the activity will be completed for this case, and this is a different transition than complete, save in the dataframe to remove this
for line in range(len(df_old)):
    if df_old.loc[line, 'lifecycle:transition'] != 'COMPLETE' and df_old.loc[line, 'case:concept:name'] in completed_dict.keys() and df_old.loc[line, 'concept:name'] in completed_dict[df_old.loc[line,'case:concept:name']]:
        df_old.loc[line, 'remove'] = 1

# Now remove the lines that are not the complete transition, but do have that for this activity.
        
df = df_old.loc[df_old['remove'] == 0].reset_index()
df = df.drop(axis = 1, columns = 'remove')
print(f'We lose {len(df_old) - len(df)} out of {len(df_old)} lines')

We lose 97632 out of 262200 lines


In [61]:
#Set up the same dataframe as we used for the activity prediction
df['next_timestamp'] = df.groupby('case:concept:name')['time:timestamp'].shift(-1)
df['next_activity'] = df.groupby('case:concept:name')['concept:name'].shift(-1)
df['previous_activity1'] = df.groupby('case:concept:name')['concept:name'].shift(1)
df['previous_activity2'] = df.groupby('case:concept:name')['concept:name'].shift(2)
le = LabelEncoder()
df['current_activity_encoded'] = le.fit_transform(df['concept:name'])
df['next_activity_encoded'] = le.fit_transform(df['next_activity'])
df['previous_activity1_encoded'] = le.fit_transform(df['previous_activity1'])
df['previous_activity2_encoded'] = le.fit_transform(df['previous_activity2'])

df['day_of_week'] = df['time:timestamp'].dt.day_name()
df['hour'] = df['time:timestamp'].dt.hour
df['day_off'] = 0
df.loc[(df['time:timestamp'].dt.year==2011) & (df['time:timestamp'].dt.month == 12) & (df['time:timestamp'].dt.day == 26), 'day_off'] = 1
df.head()

,index,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_timestamp,next_activity,previous_activity1,previous_activity2,current_activity_encoded,next_activity_encoded,previous_activity1_encoded,previous_activity2_encoded,day_of_week,hour,day_off
0,0,112,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 00:38:44.880000+00:00,A_PARTLYSUBMITTED,NaN,NaN,9,6,24,24,Saturday,0,0
1,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 00:39:37.906000+00:00,A_PREACCEPTED,A_SUBMITTED,NaN,6,7,9,24,Saturday,0,0
2,2,112,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 11:42:43.308000+00:00,A_ACCEPTED,A_PARTLYSUBMITTED,A_SUBMITTED,7,0,6,9,Saturday,0,0
3,5,10862,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 11:45:09.243000+00:00,O_SELECTED,A_PREACCEPTED,A_PARTLYSUBMITTED,0,13,7,6,Saturday,11,0
4,6,10862,COMPLETE,O_SELECTED,2011-10-01 11:45:09.243000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 11:45:09.243000+00:00,A_FINALIZED,A_ACCEPTED,A_PREACCEPTED,14,5,0,7,Saturday,11,0


In [62]:
# We now create a column that is the amount of seconds until the next activity
df['time_until_next'] = (pd.to_datetime(df['next_timestamp']) - pd.to_datetime(df['time:timestamp'])).astype('timedelta64[s]')
df['current_time_delta'] = df.groupby('case:concept:name')['time:timestamp'].diff(-1).dt.total_seconds().abs()
df.head()

,index,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_timestamp,next_activity,...,previous_activity2,current_activity_encoded,next_activity_encoded,previous_activity1_encoded,previous_activity2_encoded,day_of_week,hour,day_off,time_until_next,current_time_delta
0,0,112,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 00:38:44.880000+00:00,A_PARTLYSUBMITTED,...,NaN,9,6,24,24,Saturday,0,0,0 days 00:00:00,0.334
1,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 00:39:37.906000+00:00,A_PREACCEPTED,...,NaN,6,7,9,24,Saturday,0,0,0 days 00:00:53,53.026
2,2,112,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 11:42:43.308000+00:00,A_ACCEPTED,...,A_SUBMITTED,7,0,6,9,Saturday,0,0,0 days 11:03:05,39785.402
3,5,10862,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 11:45:09.243000+00:00,O_SELECTED,...,A_PARTLYSUBMITTED,0,13,7,6,Saturday,11,0,0 days 00:02:25,145.935
4,6,10862,COMPLETE,O_SELECTED,2011-10-01 11:45:09.243000+00:00,2011-10-01 00:38:44.546000+00:00,173688,20000,2011-10-01 11:45:09.243000+00:00,A_FINALIZED,...,A_PREACCEPTED,14,5,0,7,Saturday,11,0,0 days 00:00:00,0.000


In [63]:
# Creating a df for time predictions
# 'time:timestamp', 'case:concept:name', 'case:AMOUNT_REQ', 'next_activity', 'next_timestamp	', 'current_activity_encoded', 'next_activity_encoded', 'day_of_week', 'hour', 'time_until_next'
df_time = df.copy()
df_time = df_time.drop(['org:resource', 'lifecycle:transition', 'case:REG_DATE','next_timestamp', 'current_activity_encoded','next_activity_encoded',
                        'previous_activity1', 'previous_activity2', 'previous_activity1_encoded', 'previous_activity2_encoded'], axis=1)
df_time['weekend'] = [1 if day == 'Sunday' or day == 'Saturday' else 0 for day in df_time['day_of_week']]
df_time['night'] = [1 if hour <=8 or hour >= 21 else 0 for hour in df_time['hour']]
df_time.head()

,index,concept:name,time:timestamp,case:concept:name,case:AMOUNT_REQ,next_activity,day_of_week,hour,day_off,time_until_next,current_time_delta,weekend,night
0,0,A_SUBMITTED,2011-10-01 00:38:44.546000+00:00,173688,20000,A_PARTLYSUBMITTED,Saturday,0,0,0 days 00:00:00,0.334,1,1
1,1,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+00:00,173688,20000,A_PREACCEPTED,Saturday,0,0,0 days 00:00:53,53.026,1,1
2,2,A_PREACCEPTED,2011-10-01 00:39:37.906000+00:00,173688,20000,A_ACCEPTED,Saturday,0,0,0 days 11:03:05,39785.402,1,1
3,5,A_ACCEPTED,2011-10-01 11:42:43.308000+00:00,173688,20000,O_SELECTED,Saturday,11,0,0 days 00:02:25,145.935,1,0
4,6,O_SELECTED,2011-10-01 11:45:09.243000+00:00,173688,20000,A_FINALIZED,Saturday,11,0,0 days 00:00:00,0.000,1,0


# Reproducing activity predictions and training test split function
We will first calculate the predicted next activity for each future event, after which we will define what the training and what the test set is, as well as which columns we will be using to predict the time until the next event.

In [64]:
def train_test_splitter(df, train_size = 0.7):
    split_value = round(len(df) * train_size)

    if df.loc[split_value + 1]['time:timestamp'].hour > 6:
        test = df.loc[split_value+1:][df.loc[split_value+1:]['time:timestamp'].dt.day >= df.loc[split_value+1:]['time:timestamp'].min().day+1].sort_values(by = 'time:timestamp')
    else:
        test = df.loc[split_value + 1:]

    train = df.loc[:split_value]
    drop_set = list(set(train[train['time:timestamp'].between(test['time:timestamp'].min(), test['time:timestamp'].max())]['case:concept:name']))
    train = train[train['case:concept:name'].isin(drop_set) == False]

    # not entirely working
    overlapping_traces = set(train['case:concept:name'].unique()).intersection(set(test['case:concept:name'].unique()))
    test = test[test['case:concept:name'].isin(overlapping_traces) == False].sort_values(by = 'time:timestamp')
    train = train[train['case:concept:name'].isin(overlapping_traces) == False].sort_values(by = 'time:timestamp')

    return train, test

# Outlier removal
Here we create the function to remove values that are outliers within their activity by time_until_next.

In [65]:
def remove_outliers(df, concept_name, threshold_z = 3):

    df_temp = df.loc[df['concept:name'] == f'{concept_name}']
    
    #calculate the z-scores and set max z-score to 3.
    z = np.abs(stats.zscore(df_temp['current_time_delta']))

    #find the indices where the z-score is larger than 3.
    outlier_indices = np.where(z > threshold_z)[0]
    
    return list(outlier_indices)

In [43]:
def remove_all_outliers(df, concept_names, threshhold_z = 3):
    """This function will for each activity in concept_names remove all outliers for that activity in the given dataframe.
    """
    outlier_indices = []
    for concept_name in concept_names:
        outliers = remove_outliers(df, concept_name, threshhold_z)
        outlier_indices = outlier_indices + outliers
    df = df.drop(outlier_indices, axis=0)

    return df

In [44]:
df_new = remove_all_outliers(df_time, concept_names)

In [45]:
print(len(df_time) - len(df_new))

799


'A_ACCEPTED', \
'A_ACTIVATED', 1 value at 70, rest below 20, removed\
'A_APPROVED',\
'A_CANCELLED' 2 values above 8, at 200+ and at 160k+, 160k one is removed\
'A_DECLINED',\
'A_FINALIZED', 1 value above 55 at 240, removed\
'A_PREACCEPTED',\
'A_REGISTERED', 1 value at 2000, all others under 35, removed\
'O_ACCEPTED',\
'O_CANCELLED',1 value at 160k, all others below 55, removed\
'O_CREATED',\
'O_DECLINED',\
'O_SELECTED',\
'O_SENT',\
'O_SENT_BACK', 2 values above 5 at 75, removed\
'W_Afhandelen leads', one at 300k+, but some others around 200-250k as well and a lot up to 150k, removes all above 80k, 327/14332\
'W_Beoordelen fraude',\
'W_Completeren aanvraag',\
'W_Nabellen incomplete dossiers',\
'W_Nabellen offertes',\
'W_Valideren aanvraag', one above 8 million, all others below 2 million, removes above 250k, 125/18062\
'W_Wijzigen contractgegevens', happened 8 times in total, one was at 350k, one at 100k, and the others under 300, does not remove any\

In [67]:
remove_outliers(df_time, 'O_DECLINED')

[]

In [54]:
#We will now try to make some visualisations about this data
i = 'O_DECLINED' 
df_temp, no_outliers = remove_outliers(df_time, i)
#make a scatterplot of this new dataframe.
fig1 = px.scatter(no_outliers, x = 'time:timestamp', y = 'time_until_next', 
                  labels = {
                      'time:timestamp': 'Time (2011-2012)',
                      'case:concept:name': 'Concept name and case number',
                      'concept:name': 'Concept Name'
                },
                title = f'investigation of time dependency of activity {i}',
                width=1100, height=600)
fig1.show();
fig1 = px.scatter(df_temp, x = 'time:timestamp', y = 'time_until_next', 
                  labels = {
                      'time:timestamp': 'Time (2011-2012)',
                      'case:concept:name': 'Concept name and case number',
                      'concept:name': 'Concept Name'
                },
                title = f'investigation of time dependency of activity {i}',
                width=1100, height=600)
fig1.show();
print(df_temp.shape, no_outliers.shape)

ValueError: not enough values to unpack (expected 2, got 0)

In [49]:
def remove_outliers_vis(df, concept_name, threshold_z = 3):
    """This removes the outerliers for one category, used to make a specific visualization."""


    df_temp = df.loc[df['next_activity'] == f'{concept_name}']
    
    #calculate the z-scores and set max z-score to 3.
    z = np.abs(stats.zscore(df_temp['time_until_next']))

    #find the indices where the z-score is larger than 3.
    outlier_indices = np.where(z > threshold_z)[0]

    #make a new dataframe without the rows with outliers
    no_outliers = df_temp.drop(outlier_indices)
    
    return df_temp, no_outliers

In [50]:
#Now we investigate how much data we lost by removing outliers
lengths = {}
for concept_name in concept_names: 
    df_temp, no_outliers = remove_outliers_vis(df_time, concept_name)
    lengths[concept_name] = [len(df_temp), len(no_outliers)]


UFuncTypeError: ufunc 'multiply' cannot use operands with types dtype('<m8[s]') and dtype('<m8[s]')

In [51]:
df_lenghts = pd.DataFrame.from_dict(lengths)
df_lengths_transpose = df_lenghts.T.rename(columns={0:'Original', 1:'New'})
df_lengths_transpose['Percentage_removed'] = (df_lengths_transpose['Original'] -df_lengths_transpose['New'])/ df_lengths_transpose['Original'] * 100
df_lengths_transpose.index.name = 'Activity'

KeyError: 'Original'

In [52]:
fig1 = px.bar(df_lengths_transpose['Percentage_removed'],y= 'Percentage_removed', title='The percentage of values removed by excluding outliers',width=1100, height=600)

fig1.update_layout(title_x = 0.5)
fig1.update_layout(xaxis={'categoryorder':'total descending'})
fig1.show();

KeyError: 'Percentage_removed'

In [53]:
fig1.write_image("Outliers.png")

NameError: name 'fig1' is not defined